In [ ]:
# !git clone https://github.com/reinforcement-learning-kr/lets-do-irl.git

Cloning into 'lets-do-irl'...
remote: Enumerating objects: 1038, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1038 (delta 16), reused 15 (delta 15), pack-reused 1013
Receiving objects: 100% (1038/1038), 8.37 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (582/582), done.
Checking out files: 100% (99/99), done.


In [ ]:
!git clone https://github.com/jamin9902/cs282r-final-project.git

Cloning into 'cs282r-final-project'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 124 (delta 56), reused 77 (delta 24), pack-reused 0
Receiving objects: 100% (124/124), 9.00 MiB | 17.49 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (39/39), done.


In [ ]:
!pip install gym==0.26.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 64.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827619 sha256=8fcd163eda667f49a277a0092be3bf9f59f22924fbcb6d6095950330efd6a681
  Stored in directory: /root/.cache/pip/wheels/af/2b/30/5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import sys 
import gym 
# sys.path.append('/work/lets-do-irl/moutaincar/maxent')

In [ ]:
# from lets-do-irl.mountaincar.maxent import train
import os
import numpy as np
import random
import pickle

In [ ]:
env = gym.make('MountainCar-v0')

In [ ]:
with open("continuous", "rb") as t:
    trajectories = pickle.load(t)

In [ ]:
def idx_state(env, state):
    one_feature = 20
    env_low = env.observation_space.low
    env_high = env.observation_space.high 
    env_distance = (env_high - env_low) / one_feature
    # print("state is: ", state)
    if(isinstance(state, tuple)):
        state = state[0]
    # print("type of state: ", type(state))
    # # print(type(state[0][0]))
    # print("env low is: ", env_low[0])
    # print(state[0] - env_low[0]) 
    # print("env distance is: ", env_distance)
    position_idx = int((state[0] - env_low[0]) / env_distance[0])
    velocity_idx = int((state[1] - env_low[1]) / env_distance[1])
    state_idx = position_idx + velocity_idx * one_feature
    # print("state_idx idx_state: ", state_idx)
    return state_idx

In [ ]:
def discretize(state):
    temp = (state - env.observation_space.low) * np.array([10,100])
    return np.round(temp, 0).astype(int)

# Discretize trajectories
def discretize_trajectories(trajectories):
    discretized_trajectories = []
    for traj in trajectories: 
        discretized_traj = []
        for t in traj:
            state = t[0]
            discrete_state = discretize(state)
            discretized_traj.append((discrete_state, t[1]))
        discretized_trajectories.append(discretized_traj)
    return discretized_trajectories

In [ ]:
trajectories = discretize_trajectories(trajectories)

In [ ]:
# Add variance to state for observational ambiguity
def add_variance(state, env):
    temp = (state - env.observation_space.low) * np.array([10, 100] * np.array([np.random.normal(1, 0.05), np.random.normal(1, 0.05)]))
    if(temp[0] > 18):
        temp[0] = 18
    if(temp[1] > 14):
        temp[1] = 14
    if(temp[0] < 0):
        temp[0] = 0
    if(temp[1] < 0):
        temp[1] = 0
    return np.round(temp, 0).astype(int)

# Observational ambiguity - add variance to state for each state-action pair with probability p 
def observation_ambiguity(trajectory, env, p_suboptimal):
    for i in range(len(trajectory)):
        if random.random() < p_suboptimal:
                t = trajectory[i]
                state = add_variance(t[0], env)
                lst = list(t)
                lst[0] = state
                trajectory[i] = tuple(lst)
    return trajectory

# Action ambiguity - randomly change action for each state-action pair with probability p
def action_ambiguity(trajectory, env, p_suboptimal):
    for i in range(len(trajectory)):
        if random.random() < p_suboptimal:
                action = np.random.randint(0, env.action_space.n)
                lst = list(trajectory[i])
                lst[1] = action
                trajectory[i] = tuple(lst)
    return trajectory

# Static occlusion - remove all instances of target states
def static_occlusion(trajectory, target_states):
    remove = []
    for i in range(len(trajectory)):
        t = trajectory[i]
        state = tuple(t[0])
        for s in target_states:
            if state == s:
                remove.append(i)
    for i in range(len(remove)):
        trajectory.pop(remove[i] - i)
    return trajectory

# Dynamic occlusion - remove each state-action pair with probability p
def dynamic_occlusion(trajectory, p_suboptimal):
    remove = []
    for i in range(len(trajectory)):
        if random.random() < p_suboptimal:
            remove.append(i)
    for i in range(len(remove)):
        trajectory.pop(remove[i] - i)
    return trajectory

In [ ]:
script_path = '/work/lets-do-irl/mountaincar/maxent/train.py'

In [ ]:
x = np.load('/work/lets-do-irl/mountaincar/maxent/expert_demo/expert_demo.npy')

In [ ]:
print(x.shape)

(20, 130, 3)


In [ ]:
print(type(x))

<class 'numpy.ndarray'>


In [ ]:
# Action ambiguity - randomly change action for each state-action pair with probability p
def action_ambiguity(trajectory, p_suboptimal):
    for i in range(len(trajectory)):
        if random.random() < p_suboptimal:
                action = np.random.randint(0, 3)
                lst = list(trajectory[i])
                lst[2] = action
                trajectory[i] = lst
    return trajectory

In [ ]:
def create_ambiguous_pkl(trajectory, p_suboptimal, save_flag = False):
    ambiguous_trajectory = [action_ambiguity(trajectory[i], p_suboptimal) for i in range(len(trajectory))]
    ambiguous_trajectory = np.array(ambiguous_trajectory)
    file_path = '/work/lets-do-irl/mountaincar/maxent/expert_demo/ambiguous_expert_demo_' + str(p_suboptimal) + '.npy'
    if(save_flag):
        np.save(file_path, ambiguous_trajectory)
    return ambiguous_trajectory

In [ ]:
create_ambiguous_pkl(x, 0.9, save_flag = True)

array([[[-5.17992336e-01,  9.65191769e-04,  1.00000000e+00],
        [-5.16069191e-01,  1.92314546e-03,  0.00000000e+00],
        [-5.13202513e-01,  2.86667838e-03,  2.00000000e+00],
        ...,
        [ 6.00000000e-01,  6.71388536e-02,  1.00000000e+00],
        [ 6.00000000e-01,  6.87068589e-02,  0.00000000e+00],
        [ 6.00000000e-01,  7.00000000e-02,  0.00000000e+00]],

       [[-5.37683261e-01,  1.11394877e-03,  0.00000000e+00],
        [-5.35463710e-01,  2.21955098e-03,  2.00000000e+00],
        [-5.32155191e-01,  3.30851920e-03,  2.00000000e+00],
        ...,
        [ 6.00000000e-01,  7.00000000e-02,  0.00000000e+00],
        [ 6.00000000e-01,  7.00000000e-02,  1.00000000e+00],
        [ 6.00000000e-01,  7.00000000e-02,  0.00000000e+00]],

       [[-5.72807178e-01,  1.37794296e-03,  1.00000000e+00],
        [-5.70061512e-01,  2.74566693e-03,  0.00000000e+00],
        [-5.65968501e-01,  4.09301014e-03,  2.00000000e+00],
        ...,
        [ 6.00000000e-01,  7.00000000e-02,

In [ ]:
def dynamic_occlusion(trajectory, p_suboptimal):
    num_rows_removed = int(len(trajectory) * p_suboptimal)
    indices_removed = np.random.choice(len(trajectory), num_rows_removed, replace=False)
    trajectory = np.delete(trajectory, indices_removed, 0)
    # for i in range(len(trajectory)):
    #     if random.random() < p_suboptimal:
    #         trajectory = np.delete(trajectory, i, 0)
    #         # print(trajectory)
    return trajectory

In [ ]:
def create_dynamic_pkl(trajectory, p_suboptimal, save_flag = False):
    dynamic_trajectory = [dynamic_occlusion(trajectory[i], p_suboptimal) for i in range(len(trajectory))]
    dynamic_trajectory = np.array(dynamic_trajectory)
    file_path = '/work/lets-do-irl/mountaincar/maxent/expert_demo/dynamic_expert_demo_' + str(p_suboptimal) + '.npy'
    if(save_flag):
        np.save(file_path, dynamic_trajectory)
    return dynamic_trajectory

In [ ]:
# occluded_state_set = set()
# for i in range(len(x)):
#     occluded_state_set.add(idx_state(env, x[i]))

# # occluded_state_set = 

In [ ]:
def static_occlusion(trajectory, occluded_state_set):
    occluded_trajectory = [trajectory[i] for i in range(len(trajectory)) if i not in occluded_state_set]
    return occluded_trajectory

In [ ]:
def save_pkl(pkl_init, suboptimality_type, p_suboptimal):
    file_path = '/work/lets-do-irl/mountaincar/maxent/expert_demo/suboptimal_expert_demo_' + str(p_suboptimal) + '_' + suboptimality_type + '.npy'
    if suboptimality_type == 'ambiguous':
        pkl = create_ambiguous_pkl(pkl_init, p_suboptimal)

    np.save(file_path, pkl)
    

In [ ]:
np.save('/work/lets-do-irl/mountaincar/maxent/expert_demo/expert_demo_ambig_dynamic_4.npy', create_dynamic_pkl(create_ambiguous_pkl(x, 0.4), 0.4))

In [ ]:
# dynamic_trajectories_4
np.save('/work/lets-do-irl/mountaincar/maxent/expert_demo/dynamic_expert_demo_4.npy', dynamic_trajectories_4)

In [ ]:
suboptimal_trajectories = [action_ambiguity(x[i], 0.1) for i in range(len(x))]
suboptimal_trajectories = np.array(suboptimal_trajectories)

In [ ]:
# suboptimal_trajectories
np.save('/work/lets-do-irl/mountaincar/maxent/expert_demo/suboptimal_expert_demo_1.npy', suboptimal_trajectories)

In [ ]:
y = create_ambiguous_pkl(x, 0.4)

In [ ]:
for i in range(len(y)):
    print(y[i][:, 2])

[0. 2. 0. 0. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 0. 2. 2. 2.
 0. 1. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 2.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0. 0. 0. 1. 2.
 2. 2. 0. 0. 0. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 0. 2. 2. 1. 2. 0. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 0. 0. 1. 2. 1. 2. 0. 1.]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 0. 1. 0. 2. 2. 2.
 2. 2. 2. 1. 0. 2. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0.
 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 0. 1. 2. 2. 1. 2. 2. 0. 1. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 1. 2. 1. 2. 2. 2. 2. 0. 0. 2. 0. 0. 0. 2. 0. 2. 2. 0. 1. 2. 0. 2.
 2. 0. 2. 1. 1. 2. 2. 0. 2. 2.]
[2. 2. 2. 0. 2. 2. 1. 2. 0. 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 1. 1. 0.
 2. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 2. 0. 0. 0. 2. 1. 0. 1.

In [ ]:
x[0][0]

array([-5.17992336e-01,  9.65191769e-04,  2.00000000e+00])

In [ ]:
for i in range(len(x)):
    print(x[i][:, 2])

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.

In [ ]:
x[1][:, 2]

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [ ]:
os.system(f'python {script_path}')

0 episode score is -30494.00
1000 episode score is -578.63
2000 episode score is -408.84
3000 episode score is -347.44
4000 episode score is -313.24
5000 episode score is -288.93
6000 episode score is -274.07
7000 episode score is -260.67
8000 episode score is -251.91
9000 episode score is -242.91
10000 episode score is -235.51


0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7794d99c-2b17-4027-8f00-bed3fc160d4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>